In [1]:
import numpy as np
import skimage.io as io
import random
import os
import cv2
import pickle
import torch
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
import glob
import time

In [2]:
torch.cuda.device_count()
device = torch.device('cuda:4')
torch.cuda.empty_cache()

In [3]:
mode = 'train'

load_folder = '/home/ubuntu/COCOdataset2017/{}'.format(mode)
save_folder = '/home/ubuntu/GCC2dataset/train'
mask_folder = '{}/masks/'.format(save_folder)

batch_size = 1
input_size = (256, 256)
SMOOTH = 1e-6

In [4]:
with open('./validAnns_train.pkl', 'rb') as f:
    validAnns = pickle.load(f)

with open('./top5_dict_train_total_135737.pkl', 'rb') as f:
    top5_dict = pickle.load(f)
    
# def getImage(self, imageObj, img_folder, img_size):
#         # Read and normalize an image
#         train_img = cv2.imread(img_folder + '/' + imageObj['file_name'], cv2.IMREAD_COLOR)
#         train_img = cv2.cvtColor(train_img, cv2.COLOR_BGR2RGB)
#         train_img = cv2.resize(train_img, img_size) # (W,H)
#         return train_img


In [5]:
# 1) load all images, masks
# torch XXXX opencv operation needed

images = {}
masks = {}

start = time.time()

# images folder glob -> transform (256,256)
for i, (catId, imgId, annId) in enumerate(validAnns):
    image = cv2.imread(load_folder + '/' + str(imgId).zfill(12) + '.jpg', cv2.IMREAD_COLOR)
    image = cv2.resize(image, input_size)
    # mask = trans(mask).type(torch.bool).to(device)
    images[imgId] = image
    
    mask = cv2.imread(mask_folder + '/' + str(annId).zfill(12) + '.png', cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, input_size)
    # mask = trans(mask).type(torch.bool).to(device)
    masks[annId] = mask
    
end = time.time()

print(f"data loaded: {end - start:.5f} sec")

data loaded: 1261.52831 sec


In [6]:
print(len(images), len(masks))

86289 155737


In [7]:
masks[100000].shape

(256, 256)

In [15]:
# 2) not load images, just get images from file paths
### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

kernel = np.ones((3, 3), np.uint8)
SMOOTH = 1e-6
total = 0

for i, ((catId_1, imgId_1, annId_1), cands) in enumerate(top5_dict.items()):
    # (catId_1, imgId_1, annId_1), cands = top5_dict.items()
    start = time.time()
    img1 = images[imgId_1]
    mask1 = masks[annId_1]

    for j, (imgId_2, annId_2, _) in enumerate(cands):
        # imgId_2, annId_2, _ = cands[0]
        img2 = images[imgId_2]
        mask2 = masks[annId_2]

        # print(img1.shape, img1.dtype, mask1.shape, mask1.dtype)
        # print(img2.shape, img2.dtype, mask2.shape, mask2.dtype)
        # break

        # mask operation
        mask1_dilation = cv2.dilate(mask1, kernel, iterations=1)
        mask2_erosion = cv2.erode(mask2, kernel, iterations=1)

        intersection = cv2.bitwise_and(mask1_dilation, mask2_erosion)
        union = cv2.bitwise_or(mask1_dilation, mask2_erosion)
        diff = cv2.bitwise_xor(intersection, union)

        # print(np.min(rand_mask), np.max(rand_mask))
        invert = cv2.bitwise_not(union)//255
        # print("invert:", invert.dtype, np.min(invert), np.max(invert))

        bg = cv2.bitwise_and(img1, img1, mask = invert) 
        fg = cv2.bitwise_and(img1, img1, mask = intersection)
        # com = cv2.copyTo(fg, mask, bg.copy())

        # check before imwrite !!

        # to all three-channel images (RGB->BGR)
        # bg = cv2.cvtColor(bg, cv2.COLOR_RGB2BGR)
        # fg = cv2.cvtColor(fg, cv2.COLOR_RGB2BGR)
        # img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2BGR)
        # com = cv2.cvtColor(com, cv2.COLOR_RGB2BGR)

        cv2.imwrite(save_folder + '/alltypes/' + str(imgId_1).zfill(12) + '_' + str(annId_1).zfill(12) + '_' + str(imgId_2).zfill(12) + '_' + str(annId_2).zfill(12) + '_' + str(j) + '_bg.png', bg)
        cv2.imwrite(save_folder + '/alltypes/' + str(imgId_1).zfill(12) + '_' + str(annId_1).zfill(12) + '_' + str(imgId_2).zfill(12) + '_' + str(annId_2).zfill(12) + '_' + str(j) + '_diff.png', diff)
        cv2.imwrite(save_folder + '/alltypes/' + str(imgId_1).zfill(12) + '_' + str(annId_1).zfill(12) + '_' + str(imgId_2).zfill(12) + '_' + str(annId_2).zfill(12) + '_' + str(j) + '_fg.png', fg)
        cv2.imwrite(save_folder + '/alltypes/' + str(imgId_1).zfill(12) + '_' + str(annId_1).zfill(12) + '_' + str(imgId_2).zfill(12) + '_' + str(annId_2).zfill(12) + '_' + str(j) + '_gt.png', img1)
        cv2.imwrite(save_folder + '/alltypes/' + str(imgId_1).zfill(12) + '_' + str(annId_1).zfill(12) + '_' + str(imgId_2).zfill(12) + '_' + str(annId_2).zfill(12) + '_' + str(j) + '_mask.png', mask1)
                

    end = time.time()
    elapsed = end - start
    total += elapsed

    print("[{}/{}]".format(i, len(top5_dict)), f"average time: {total/(i+SMOOTH):.5f} sec", end='\r')
    
# f, ax = plt.subplots(nrows=1, ncols=5, figsize=(8, 8))
# f.tight_layout()

# ax[0].imshow(mask1.squeeze())
# ax[0].set_title("mask1")
# ax[0].axis('off')

# ax[1].imshow(fg.squeeze())
# ax[1].set_title("fg")
# ax[1].axis('off')

# ax[2].imshow(intersection.squeeze())
# ax[2].set_title("intersection")
# ax[2].axis('off')

# ax[3].imshow(union.squeeze())
# ax[3].set_title("union")
# ax[3].axis('off')

# ax[4].imshow(diff.squeeze())
# ax[4].set_title("diff")
# ax[4].axis('off')

  

# image save with title 'annId_imgtype_i' (including 0)
# warning: RGB BGR before save
# imgtype: [bg, fg, gt, mask]


